# TP Noté : Robot Learning - Manipulation Robotique Robot So-Arm 101 – HuggingFace LeRobot 🤗🤖

Mineure Robotique - Décembre 2025

## Membres du groupe
- Alexis Le Trung
- Matéo Lelong
- Maxim Bocquillion
- Titouan Verhille

# Contexte et objectifs du TP

Le but de ce projet est la manipulation du robot So-Arm 101 et la mise en place d'une pipeline complète de Robot Learning par imitation.

<img src="./assets/robots.png" alt="robots" width="500">

# Configuration et imports

In [ ]:
# import 

# Installation et calibration

La première étape de ce projet est **la calibration** les deux robots.

Voici les résultats obtenus après calibration sur les **positions minimales et maximales** des robots :
<img src="./assets/calibration.png" alt="robots" width="700">

# Chargement et analyse des données brutes

# Nettoyage et augmentation des données

# Création et validation du dataset LeRobot

# Entraînement de la politique

# Tests du modèle et résultats

# Analyse critique et conclusions